In [1]:
## Import necessary libraries
import pandas as pd
import ast
import missingno as msno
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
## Define file locations
TRACK_FILE = '../datasets/fma_metadata/tracks.csv'
GENRE_FILE = '../datasets/fma_metadata/genres.csv'

In [5]:
## Read tracks file, change column names, remove unnecessary rows
track_df = pd.read_csv(TRACK_FILE, header=[0,1])
new_columns = ['_'.join(col).strip() for col in track_df.columns.values]
track_df.columns = new_columns
track_df = track_df.rename(columns={'Unnamed: 0_level_0_Unnamed: 0_level_1': 'track_id'})
track_df = track_df.iloc[1:]

/var/folders/np/f11s751j4dj_y_1cj4yxydpwpsr474/T/ipykernel_38277/3781808237.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  track_df = pd.read_csv(TRACK_FILE, header=[0,1])


In [7]:
## Read genres file
genre_df = pd.read_csv(GENRE_FILE)

In [8]:
## Create directed graph (Tree) of genre
G = nx.DiGraph()

for index, row in genre_df.iterrows():
    G.add_node(row['genre_id'], label=row['title'])
    
for index, row in genre_df.iterrows():
    if pd.notna(row['parent']):
        G.add_edge(row['parent'], row['genre_id'])

In [38]:
def get_lineage(node, G, genre_df):
    lineage_ids = [node]
    lineage_names = [genre_df[genre_df['genre_id'] == node]['title'].values[0]]
    while list(G.predecessors(node)):
        node = list(G.predecessors(node))[0]
        lineage_ids.append(node)
        lineage_names.append(genre_df[genre_df['genre_id'] == node]['title'].values[0])
    return lineage_ids[::-1], lineage_names[::-1]  # Return in root-to-leaf order


In [39]:
lineages_ids = {}
lineages_names = {}
for genre in G.nodes():
    if genre == 0:
        continue;
    ids, names = get_lineage(genre, G, genre_df)
    lineages_ids[genre] = ids
    lineages_names[genre] = names

# Transform lineage into DataFrame
lineage_df_ids = pd.DataFrame.from_dict(lineages_ids, orient='index').reset_index()
lineage_df_names = pd.DataFrame.from_dict(lineages_names, orient='index').reset_index()
lineage_df_ids.columns = ['genre_id'] + [f'depth_{i+1}_genre_id' for i in range(lineage_df_ids.shape[1]-1)]
lineage_df_names.columns = ['genre_id'] + [f'depth_{i+1}_genre_name' for i in range(lineage_df_names.shape[1]-1)]

# Handle missing depths
for i in range(2, lineage_df_ids.shape[1]):
    lineage_df_ids.iloc[:, i].fillna(lineage_df_ids.iloc[:, i-1], inplace=True)
    lineage_df_names.iloc[:, i].fillna(lineage_df_names.iloc[:, i-1], inplace=True)


TypeError: cannot unpack non-iterable NoneType object

In [18]:
lineage_df

,genre_id,depth_1_genre,depth_2_genre,depth_3_genre,depth_4_genre,depth_5_genre,depth_6_genre
0,1,0,38.0,1.0,1.0,1.0,1.0
1,2,0,2.0,2.0,2.0,2.0,2.0
2,3,0,3.0,3.0,3.0,3.0,3.0
3,4,0,4.0,4.0,4.0,4.0,4.0
4,5,0,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...
159,1060,0,2.0,46.0,1060.0,1060.0,1060.0
160,1156,0,2.0,130.0,1156.0,1156.0,1156.0
161,1193,0,38.0,6.0,16.0,763.0,1193.0
162,1235,0,1235.0,1235.0,1235.0,1235.0,1235.0


In [11]:
## Transfrom the track_df so that the every single row belongs to only one genre
track_df['track_genres_lst'] = track_df['track_genres'].apply(ast.literal_eval)
track_df_single_genre = track_df.explode('track_genres_lst')

In [13]:
## Merge genre information into tracks
merged_df = track_df_single_genre.merge(leaf_df, left_on='track_genres_lst', right_on='leaf_id', how='left')

In [ ]:
## 

In [15]:
merged_df.to_csv('./datasets/tracks_with_genre.csv')